**Makes a new feature class of parcels that allow single family homes then calculates how many lots each parcel could legally have based on the Land Development Code minimum lot size. Then creates another feature class from that for only those within the City limits.**

City Limits Feature Class is current as of: 9/7/2022

**Notes:**
- did not include Rural, Lake Protection, or Urban Fringe
- R4 merely states "Cannot exceed the maximum density of ten units per acre" so is not included outside the MMTD.
- MCN, UP-1 & UP-2 do not have minimum lot size standards outside the MMTD
- for lots in the MMTD, the standards provided minimum width & depth, which was used to calculate min size.

** IMPORTANT REMINDER: ** Once the feature layer has been pushed to the web, the two feature classes created here were renamed with a 'TEMP' prefix. To use in ArcPro, delete all the records from the previous feature class & copy/paste the new ones in. This preserves Web application connections. 

In [11]:
###############################################
# [1] SET ENVIRONMENT & SOURCE LOCATION 
###############################################

#import necessary libraries
import arcpy

# set the workspace
arcpy.env.workspace = r"C:\Users\bryantch\Documents\ArcGIS\Projects\LotSizeAnalysis\LotSizeAnalysis.gdb"

# tell python it's OK to overwrite previous versions of layers & feature classes
arcpy.env.overwriteOutput = True

# create variable for source feature class
source_fc = "C:/Users/bryantch/Documents/Data/exlu2022detail.gdb/exlu2022detail"

In [12]:
###############################################
# [2] SELECT ONLY THOSE PARCELS WITH ZONING THAT PRIMARILY ALLOWS SINGLE FAMILY HOMES
###############################################

try:
    out_table = "single_fam_zoned_parcels"

    # make expression to select districts with appropriate zoning
    # and remove government, education & railroad parcels
    where_clause1 = "ZONING in ('RP-1', 'RP-2', 'RP-MH', 'RA', 'R-1', 'R-2', 'R-3', 'R-4', 'R-5', \
        'MR-1', 'OR-1', 'OR-2', 'OR-3', 'NBO', 'MR', 'NB-1', 'RP-1', 'RP-2', 'CU-12', 'UP-1', 'CU-18', \
        'CU-26', 'UP-2', 'CU-45', 'UT', 'ASN-A', 'ASN-B', 'ASN-C', 'ASN-D', 'UV', 'MCN') \
        AND OWNER1 not in ('CITY OF TALLAHASSEE','LEON COUNTY','LEON COUNTY SCHOOL BOARD','LEON COUNTY EDUCATIONAL FACILITIES AUTH.','LEON COUNTY EDUCATIONAL',\
        'SCHOOL BOARD LEON COUNTY','SCHOOL BOARD OF LEON COUNTY','FLORIDA STATE UNIVERSITY BOARD OF', 'FLORIDA STATE UNIVERSITY BOARD TRUSTEES', \
        'FLORIDA STATE UNIVERSITY RESEARCH', 'FLORIDA STATE UNIVERSITY', 'FLORIDA STATE UNIVERSITY BOARD', 'FLORIDA GAME & FISH COMMISSION', \
        'FLORIDA A & M UNIV BD OF TRUSTEES', 'FLORIDA A & M UNIVERSITY', 'FLORIDA A&M UNIVERSITY FOUNDATION INC', 'TALLAHASSEE COMMUNITY', \
        'TALLAHASSEE COMMUNITY COLLEGE', 'TALLAHASSEE COMMUNITY COLLEGE FLORIDA', 'FLORIDA DEPARTMENT OF TRANSPORTATION', 'STATE OF FLORIDA',\
        'TIITF','UNITED STATES OF AMERICA', 'FLORIDA GULF & ATLANTIC RAILROAD LLC')"     
       
    # run Select tool & create a new feature class called 'correct_zoned' with the results
    # use 'analysis.Select' instead of 'management.SelectLayerByAttribute' b/c 'Select' automatically writes a
        # new feature class to .gdb and we need a permanent feature class
    arcpy.analysis.Select(source_fc, out_table, where_clause1)
    
except: 
    print(arcpy.GetMessages())

In [14]:
###############################################
# [3] CALCULATE HOW MANY LOTS EACH PARCEL COULD LEGALLY ACCOMODATE & SAVE AS A NEW FIELD
###############################################

try:
    inTable_lots = "single_fam_zoned_parcels"
    fieldName_lots = "lots_allowed"
    expression_lots = "calc_lots_allowed(!ZONING!, !CALC_ACREA!, !mmtd!)"

    # function to calculate the lots allowed per parcel based on minimum lot size (per Land Development Code)
    # provides separate calculations based on being in or out of MMMTD
    codeblock_lots = '''
def calc_lots_allowed(zoning, parcel_acreage, mmtd):
    if (mmtd == 'Out' and zoning == 'RP-1'):
        return parcel_acreage/.138
    elif (mmtd == 'Out' and zoning == 'RP-2'):
        return parcel_acreage/.138
    elif (mmtd == 'Out' and zoning == 'RP-MH'):
        return parcel_acreage/.083
    elif (mmtd == 'Out' and zoning == 'RA'):
        return parcel_acreage/1
    elif (mmtd == 'Out' and zoning == 'R-1'):
        return parcel_acreage/.275
    elif (mmtd == 'Out' and zoning == 'R-2'):
        return parcel_acreage/.207
    elif (mmtd == 'Out' and zoning == 'R-3'):
        return parcel_acreage/.115
    elif (mmtd == 'Out' and zoning == 'R-5'):
        return parcel_acreage/.115
    elif (mmtd == 'Out' and zoning == 'MR-1'):
        return parcel_acreage/.115
    elif (mmtd == 'Out' and zoning == 'OR-1'):
        return parcel_acreage/.115
    elif (mmtd == 'Out' and zoning == 'OR-2'):
        return parcel_acreage/.115
    elif (mmtd == 'Out' and zoning == 'OR-3'):
        return parcel_acreage/.115
    elif (mmtd == 'Out' and zoning == 'NBO'):
        return parcel_acreage/.115
    elif (mmtd == 'Out' and zoning == 'MR'):
        return parcel_acreage/.115
    elif (mmtd == 'Out' and zoning == 'NB-1'):
        return parcel_acreage/.024
    elif (mmtd == 'In' and zoning == 'RP-1'):
        return parcel_acreage/.024
    elif (mmtd == 'In' and zoning == 'RP-2'):
        return parcel_acreage/.024
    elif (mmtd == 'In' and zoning == 'R-1'):
        return parcel_acreage/.024
    elif (mmtd == 'In' and zoning == 'R-2'):
        return parcel_acreage/.024
    elif (mmtd == 'In' and zoning == 'R-3'):
        return parcel_acreage/.02
    elif (mmtd == 'In' and zoning == 'R-5'):
        return parcel_acreage/.024
    elif (mmtd == 'In' and zoning == 'NBO'):
        return parcel_acreage/.02
    elif (mmtd == 'In' and zoning == 'NB-1'):
        return parcel_acreage/.024
    elif (mmtd == 'In' and zoning == 'OR-1'):
        return parcel_acreage/.024
    elif (mmtd == 'In' and zoning == 'CU-12'):
        return parcel_acreage/.024
    elif (mmtd == 'In' and zoning == 'R-4'):
        return parcel_acreage/.019
    elif (mmtd == 'In' and zoning == 'OR-2'):
        return parcel_acreage/.019
    elif (mmtd == 'In' and zoning == 'UP-1'):
        return parcel_acreage/.019
    elif (mmtd == 'In' and zoning == 'MR-1'):
        return parcel_acreage/.019
    elif (mmtd == 'In' and zoning == 'CU-18'):
        return parcel_acreage/.019
    elif (mmtd == 'In' and zoning == 'CU-26'):
        return parcel_acreage/.019
    elif (mmtd == 'In' and zoning == 'MR'):
        return parcel_acreage/.019
    elif (mmtd == 'In' and zoning == 'OR-3'):
        return parcel_acreage/.015
    elif (mmtd == 'In' and zoning == 'UP-2'):
        return parcel_acreage/.015
    elif (mmtd == 'In' and zoning == 'CU-45'):
        return parcel_acreage/.015
    elif (mmtd == 'In' and zoning == 'UT'):
        return parcel_acreage/.015
    elif (mmtd == 'In' and zoning == 'ASN-A'):
        return parcel_acreage/.015
    elif (mmtd == 'In' and zoning == 'ASN-B'):
        return parcel_acreage/.015
    elif (mmtd == 'In' and zoning == 'ASN-C'):
        return parcel_acreage/.015
    elif (mmtd == 'In' and zoning == 'ASN-D'):
        return parcel_acreage/.015
    elif (mmtd == 'In' and zoning == 'UV'):
        return parcel_acreage/.015
    '''
 
    # calculate the new field
    arcpy.management.CalculateField(inTable_lots, fieldName_lots, expression_lots, "PYTHON3", codeblock_lots, field_type="DOUBLE")

except: 
    print(arcpy.GetMessages())

In [15]:
###############################################
# [4] CALCULATE HOW MANY UNITS EACH PARCEL COULD LEGALLY ACCOMODATE & SAVE AS A NEW FIELD
###############################################

try:

    # set tool paramaters
    inTable_parcels = "single_fam_zoned_parcels"
    fieldName_parcels = "units_allowed"
    expression_parcels = "calc_units_allowed(!ZONING!, !CALC_ACREA!)"

    # create a dictionary of zoning districts & their max allowed density
    densityDict = {
        'ASN-A': 50,
        'ASN-B': 50,
        'ASN-C': 75,
        'ASN-D': 100,
        'CU-12': 12,
        'CU-18': 18,
        'CU-26': 26,
        'CU-45': 45,
        'LP': 0.5,
        'MCN': 12,
        'MH': 8,
        'MR-1': 20,
        'NB-1': 18,
        'NBO': 8,
        'OR-1': 8,
        'OR-2': 16,
        'OR-3': 20,
        'RA': 1,
        'R-1': 3.63,
        'R-2': 4.84,
        'R-3': 8,
        'R-4': 10,
        'R-5': 8,
        'R': 0.1,
        'RP-1': 3.6,
        'RP-2': 6,
        'RP-MH': 6,
        'UP-1': 16,
        'UP-2': 20,
        'UT': 50,
        'UV': 100
        }

    # function to calculate the units allowed per parcel based on lot agreage (per Land Development Code)
    codeblock_parcels = """ 
def calc_units_allowed(zoning, parcel_acreage):
    for district in densityDict:
        if zoning == district:
            return parcel_acreage * densityDict[district]"""

    # calculate the new field
    arcpy.management.CalculateField(inTable_parcels, fieldName_parcels, expression_parcels, "PYTHON3", codeblock_parcels, field_type="DOUBLE")

except: 
    print(arcpy.GetMessages())

In [16]:
##########################################################
# [5] CLIP THE ABOVE NEWLY CREATED FC BY CITY LIMITS & SAVE AS NEW FC
##########################################################
try: 

    in_features = "single_fam_zoned_parcels"
    clip_features = r"C:\Users\bryantch\Documents\Data\CityLimits_9_7_22\City_Limits_for_Tallahassee%2C_Florida.shp"
    out_feature_class = "TEMP_single_fam_zoned_parcels_CITY"

    arcpy.analysis.Clip(in_features, clip_features, out_feature_class)

except: 
    print(arcpy.GetMessages())